In [1]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [2]:
# Clear any logs from previous runs
!rm -rf ./logs/

In [3]:
!pip install hparams

In [4]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np
from tensorboard.plugins.hparams import api as hp

In [5]:
fashion_mnist = tf.keras.datasets.fashion_mnist
(x_train, y_train),(x_test, y_test) = fashion_mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

In [6]:
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test =x_test.reshape(x_test.shape[0], 28, 28, 1)

In [7]:
HP_ACTIVATION_1 = hp.HParam('activation1', hp.Discrete(['relu', 'selu', 'elu']))
HP_ACTIVATION_2 = hp.HParam('activation2', hp.Discrete(['relu', 'selu', 'elu']))
HP_ACTIVATION_3 = hp.HParam('activation3', hp.Discrete(['relu', 'selu', 'elu']))
HP_ACTIVATION_4 = hp.HParam('activation4', hp.Discrete(['relu', 'selu', 'elu']))
METRIC_ACCURACY = 'accuracy'
with tf.summary.create_file_writer('logs/hparam_tuning').as_default():
  hp.hparams_config(hparams=[HP_ACTIVATION_1, HP_ACTIVATION_2, HP_ACTIVATION_3, HP_ACTIVATION_4], metrics=[hp.Metric(METRIC_ACCURACY, display_name='Accuracy')])

In [8]:
def train_test_model(hparams):
  model = keras.Sequential([
    keras.layers.Conv2D(32, kernel_size = (5,5), padding = 'same', input_shape = (28,28,1)),
    keras.layers.BatchNormalization(),
    keras.layers.Activation(hparams[HP_ACTIVATION_1]),
    keras.layers.Conv2D(32, kernel_size = (5,5), padding = 'same'),
    keras.layers.BatchNormalization(),
    keras.layers.Activation(hparams[HP_ACTIVATION_2]),
    keras.layers.MaxPooling2D(pool_size = (2, 2)),
    keras.layers.Dropout(0.30),
    keras.layers.Conv2D(64, [3, 3],padding ='same'),
    keras.layers.BatchNormalization(),
    keras.layers.Activation(hparams[HP_ACTIVATION_3]),
    keras.layers.Conv2D(64, [3, 3],padding ='same'),
    keras.layers.BatchNormalization(),
    keras.layers.Activation(hparams[HP_ACTIVATION_4]),
    keras.layers.MaxPooling2D(pool_size=(2, 2)),
    keras.layers.Dropout(0.30),
    keras.layers.Flatten(),
    keras.layers.Dense(1024, activation ='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(512, activation ='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.30),
    keras.layers.Dense(10, activation ='softmax'),
])
  model.compile(optimizer='Nadam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

  model.fit(x_train, y_train, epochs=10) # Run with 1 epoch to speed things up for demo purposes
  _, accuracy = model.evaluate(x_test, y_test)
  return accuracy

In [9]:
Accuracy = []
combination = 0
def run(run_dir, hparams):
  with tf.summary.create_file_writer(run_dir).as_default():
    global combination
    combination = combination + 1
    hp.hparams(hparams)  # record the values used in this trial
    accuracy = train_test_model(hparams)
    Accuracy.append(['combination'+str(combination),list(hparams.values())[0],list(hparams.values())[1],list(hparams.values())[2],list(hparams.values())[3],accuracy])
    tf.summary.scalar(METRIC_ACCURACY, accuracy, step=1)

In [10]:
session_num = 0
# for num_units in HP_NUM_UNITS.domain.values:
# for dropout_rate in (HP_DROPOUT.domain.min_value, HP_DROPOUT.domain.max_value):
for activation1 in HP_ACTIVATION_1.domain.values:
  for activation2 in HP_ACTIVATION_2.domain.values:
    for activation3 in HP_ACTIVATION_3.domain.values:
      for activation4 in HP_ACTIVATION_4.domain.values:
        hparams = { HP_ACTIVATION_1: activation1, HP_ACTIVATION_2: activation2, HP_ACTIVATION_3: activation3, HP_ACTIVATION_4: activation4}
        run_name = activation1
        print('--- Starting trial: %s' % run_name)
        print({h.name: hparams[h] for h in hparams})
        run('logs/hparam_tuning/' + run_name, hparams)      
        session_num += 1

--- Starting trial: elu
{'activation1': 'elu', 'activation2': 'elu', 'activation3': 'elu', 'activation4': 'elu'}
Epoch 1/10
1875/1875 [==============================] - 17s 7ms/step - loss: 0.4766 - accuracy: 0.8329
Epoch 2/10
1875/1875 [==============================] - 14s 7ms/step - loss: 0.3285 - accuracy: 0.8807
Epoch 3/10
1875/1875 [==============================] - 14s 7ms/step - loss: 0.2878 - accuracy: 0.8944
Epoch 4/10
1875/1875 [==============================] - 14s 7ms/step - loss: 0.2591 - accuracy: 0.9054
Epoch 5/10
1875/1875 [==============================] - 13s 7ms/step - loss: 0.2388 - accuracy: 0.9129
Epoch 6/10
1875/1875 [==============================] - 13s 7ms/step - loss: 0.2195 - accuracy: 0.9196
Epoch 7/10
1875/1875 [==============================] - 13s 7ms/step - loss: 0.1997 - accuracy: 0.9253
Epoch 8/10
1875/1875 [==============================] - 13s 7ms/step - loss: 0.1854 - accuracy: 0.9325
Epoch 9/10
1875/1875 [==============================] - 13s 7ms

In [11]:
df = pd.DataFrame(Accuracy,columns=('combination_no:','Activation1','Activation2','Activation3','Activation4', 'Accuracy'))
df
with pd.option_context('display.max_rows', None, 'display.max_columns', None): 
    display(df)
column = df["Accuracy"]
max_value = column.idxmax()

,combination_no:,Activation1,Activation2,Activation3,Activation4,Accuracy
0,combination1,elu,elu,elu,elu,0.9299
1,combination2,elu,elu,elu,relu,0.9134
2,combination3,elu,elu,elu,selu,0.9277
3,combination4,elu,elu,relu,elu,0.9320
4,combination5,elu,elu,relu,relu,0.9309
5,combination6,elu,elu,relu,selu,0.9279
6,combination7,elu,elu,selu,elu,0.9232
7,combination8,elu,elu,selu,relu,0.9246
8,combination9,elu,elu,selu,selu,0.9219
9,combination10,elu,relu,elu,elu,0.9275


In [12]:
print("The best model combination is displayed as follows:\n")
df.iloc[max_value,:]

The best model combination is displayed as follows:



combination_no:    combination65
Activation1                 selu
Activation2                 relu
Activation3                  elu
Activation4                 relu
Accuracy                  0.9341
Name: 64, dtype: object